In [2]:
# import packages for data processing
import pandas as pd
import numpy as np
from sklearn.preprocessing import PowerTransformer

# import packages for data visualisation
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

# import packages for statistical analysis
from scipy import stats
from scipy.stats import chi2_contingency

# import packages for model building


# set display options
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

In [3]:
# load training data
df_train = pd.read_csv('dataset/train.csv')

In [4]:
df_train

,FILENAME,URL,URLLength,Domain,DomainLength,IsDomainIP,TLD,URLSimilarityIndex,CharContinuationRate,TLDLegitimateProb,URLCharProb,TLDLength,NoOfSubDomain,HasObfuscation,NoOfObfuscatedChar,ObfuscationRatio,NoOfLettersInURL,LetterRatioInURL,NoOfDegitsInURL,DegitRatioInURL,NoOfEqualsInURL,NoOfQMarkInURL,NoOfAmpersandInURL,NoOfOtherSpecialCharsInURL,SpacialCharRatioInURL,IsHTTPS,LineOfCode,LargestLineLength,HasTitle,Title,DomainTitleMatchScore,URLTitleMatchScore,HasFavicon,Robots,IsResponsive,NoOfURLRedirect,NoOfSelfRedirect,HasDescription,NoOfPopup,NoOfiFrame,HasExternalFormSubmit,HasSocialNet,HasSubmitButton,HasHiddenFields,HasPasswordField,Bank,Pay,Crypto,HasCopyrightInfo,NoOfImage,NoOfCSS,NoOfJS,NoOfSelfRef,NoOfEmptyRef,NoOfExternalRef,label
0,8013435.txt,https://api.auth.gg/,19,api.auth.gg,11,0,gg,66.666667,0.625000,0.000178,0.046504,2,1,0,0,0.0,9,0.474,0,0.000,0,0,0,2,0.105,1,61,431,1,apiauth,0.0,0.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,6,1
1,233757.txt,https://www.polisci.upenn.edu,28,www.polisci.upenn.edu,21,0,edu,100.000000,0.615385,0.011501,0.057601,3,2,0,0,0.0,14,0.500,0,0.000,0,0,0,2,0.071,1,1228,1833,1,polisciupenn,0.0,0.0,0,0,1,0,0,0,0,1,0,0,1,1,0,0,0,0,1,18,4,7,89,0,15,0
2,409841.txt,https://www.oscarhunt.com.au,27,www.oscarhunt.com.au,20,0,au,100.000000,0.769231,0.010086,0.066248,2,2,0,0,0.0,13,0.481,0,0.000,0,0,0,2,0.074,1,1679,1867,1,oscrhuntcom,0.0,0.0,1,1,1,0,0,1,0,2,0,1,1,1,0,0,0,0,1,32,3,11,72,0,63,0
3,552154.txt,https://www.crowncabinets.com,28,www.crowncabinets.com,21,0,com,100.000000,1.000000,0.522907,0.065279,3,1,0,0,0.0,15,0.536,0,0.000,0,0,0,1,0.036,1,967,1981,1,crowncabinets,100.0,100.0,0,0,1,0,0,1,0,2,0,1,0,1,0,0,0,0,1,6,8,17,44,2,16,0
4,482480.txt,https://www.robertlpeters.com,28,www.robertlpeters.com,21,0,com,100.000000,1.000000,0.522907,0.065927,3,1,0,0,0.0,15,0.536,0,0.000,0,0,0,1,0.036,1,135,650,1,robertlpeters,100.0,100.0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,24,4,2,0,0,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165051,149524.txt,https://www.thejusticegap.com,28,www.thejusticegap.com,21,0,com,100.000000,1.000000,0.522907,0.056816,3,1,0,0,0.0,15,0.536,0,0.000,0,0,0,1,0.036,1,2645,9381,1,thejusticegap,100.0,100.0,0,1,1,0,0,0,0,0,1,1,1,1,0,0,0,0,1,26,38,46,160,1,167,0
165052,682868.txt,https://www.millisavunma.com,27,www.millisavunma.com,20,0,com,100.000000,1.000000,0.522907,0.059284,3,1,0,0,0.0,14,0.519,0,0.000,0,0,0,1,0.037,1,3671,9381,1,millisavunma,100.0,100.0,0,1,1,0,0,1,0,14,0,1,0,1,0,0,1,0,1,99,16,23,251,3,294,0
165053,8077146.txt,https://hzlt1adwk.firebaseapp.com/,34,hzlt1adwk.firebaseapp.com,25,0,com,71.212308,0.619048,0.522907,0.048478,3,1,0,0,0.0,22,0.647,1,0.029,0,0,0,3,0.088,1,2,2281,1,hzlt1adwkfirebaseapp,0.0,0.0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,1,0,0,1
165054,8117980.txt,http://globalmetamask.com,25,globalmetamask.com,18,0,com,78.037687,1.000000,0.522907,0.062371,3,0,0,0,0.0,17,0.680,0,0.000,0,0,0,1,0.040,0,9,77,1,globalmetamask,100.0,100.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1


In [5]:
def preprocess_data(df):
    # drop FILENAME column
    df = df.drop(columns=['FILENAME'])
    
    # convert boolean columns to 'bool' data type
    bool_columns = ['IsDomainIP', 'HasObfuscation', 'IsHTTPS', 'HasTitle', 'HasFavicon',
                    'Robots', 'IsResponsive', 'HasDescription', 'HasExternalFormSubmit',
                    'HasSocialNet', 'HasSubmitButton', 'HasHiddenFields','HasPasswordField',
                    'Bank', 'Pay', 'Crypto', 'HasCopyrightInfo']
    for col in bool_columns:
        df[col] = df[col].astype(bool)
    
    return df
df_train = preprocess_data(df_train)

In [7]:
text_variables = df_train.select_dtypes(include=['object']).columns.tolist() # exclude the text fields
df_train = df_train.drop(columns=text_variables)

In [8]:
df_train

,URLLength,DomainLength,IsDomainIP,URLSimilarityIndex,CharContinuationRate,TLDLegitimateProb,URLCharProb,TLDLength,NoOfSubDomain,HasObfuscation,NoOfObfuscatedChar,ObfuscationRatio,NoOfLettersInURL,LetterRatioInURL,NoOfDegitsInURL,DegitRatioInURL,NoOfEqualsInURL,NoOfQMarkInURL,NoOfAmpersandInURL,NoOfOtherSpecialCharsInURL,SpacialCharRatioInURL,IsHTTPS,LineOfCode,LargestLineLength,HasTitle,DomainTitleMatchScore,URLTitleMatchScore,HasFavicon,Robots,IsResponsive,NoOfURLRedirect,NoOfSelfRedirect,HasDescription,NoOfPopup,NoOfiFrame,HasExternalFormSubmit,HasSocialNet,HasSubmitButton,HasHiddenFields,HasPasswordField,Bank,Pay,Crypto,HasCopyrightInfo,NoOfImage,NoOfCSS,NoOfJS,NoOfSelfRef,NoOfEmptyRef,NoOfExternalRef,label
0,19,11,False,66.666667,0.625000,0.000178,0.046504,2,1,False,0,0.0,9,0.474,0,0.000,0,0,0,2,0.105,True,61,431,True,0.0,0.0,False,False,True,0,0,False,0,0,False,False,False,False,False,False,False,False,True,0,1,0,0,0,6,1
1,28,21,False,100.000000,0.615385,0.011501,0.057601,3,2,False,0,0.0,14,0.500,0,0.000,0,0,0,2,0.071,True,1228,1833,True,0.0,0.0,False,False,True,0,0,False,0,1,False,False,True,True,False,False,False,False,True,18,4,7,89,0,15,0
2,27,20,False,100.000000,0.769231,0.010086,0.066248,2,2,False,0,0.0,13,0.481,0,0.000,0,0,0,2,0.074,True,1679,1867,True,0.0,0.0,True,True,True,0,0,True,0,2,False,True,True,True,False,False,False,False,True,32,3,11,72,0,63,0
3,28,21,False,100.000000,1.000000,0.522907,0.065279,3,1,False,0,0.0,15,0.536,0,0.000,0,0,0,1,0.036,True,967,1981,True,100.0,100.0,False,False,True,0,0,True,0,2,False,True,False,True,False,False,False,False,True,6,8,17,44,2,16,0
4,28,21,False,100.000000,1.000000,0.522907,0.065927,3,1,False,0,0.0,15,0.536,0,0.000,0,0,0,1,0.036,True,135,650,True,100.0,100.0,False,False,False,0,0,False,0,0,False,True,False,False,False,False,False,False,True,24,4,2,0,0,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165051,28,21,False,100.000000,1.000000,0.522907,0.056816,3,1,False,0,0.0,15,0.536,0,0.000,0,0,0,1,0.036,True,2645,9381,True,100.0,100.0,False,True,True,0,0,False,0,0,True,True,True,True,False,False,False,False,True,26,38,46,160,1,167,0
165052,27,20,False,100.000000,1.000000,0.522907,0.059284,3,1,False,0,0.0,14,0.519,0,0.000,0,0,0,1,0.037,True,3671,9381,True,100.0,100.0,False,True,True,0,0,True,0,14,False,True,False,True,False,False,True,False,True,99,16,23,251,3,294,0
165053,34,25,False,71.212308,0.619048,0.522907,0.048478,3,1,False,0,0.0,22,0.647,1,0.029,0,0,0,3,0.088,True,2,2281,True,0.0,0.0,False,False,True,0,0,False,0,0,False,False,False,True,False,False,False,False,False,0,1,1,1,0,0,1
165054,25,18,False,78.037687,1.000000,0.522907,0.062371,3,0,False,0,0.0,17,0.680,0,0.000,0,0,0,1,0.040,False,9,77,True,100.0,100.0,False,False,False,0,0,False,0,0,False,False,False,False,False,False,False,False,False,0,0,0,1,0,1,1


### Run a Log Regression for each feature

In [22]:
# Single-feature and swap tests to find potential leakage features

def run_log_single(df, target='label', predictors=None, test_size=0.2, random_state=42):

    from sklearn.model_selection import train_test_split
    from sklearn.linear_model import LogisticRegression
    from sklearn.preprocessing import StandardScaler
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

    if predictors is None:
        predictors = [c for c in df.columns if c != target]

    y = df[target].copy()
    results = []
    for feat in predictors:
        X = df[[feat]].copy()
        X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=test_size, stratify=y, random_state=random_state)
        scaler = StandardScaler()
        X_tr_s = scaler.fit_transform(X_tr)
        X_te_s = scaler.transform(X_te)
        m = LogisticRegression(solver='liblinear', max_iter=2000)
        m.fit(X_tr_s, y_tr)
        y_pred = m.predict(X_te_s)
        probs = m.predict_proba(X_te_s)[:, 1] if m.classes_.shape[0] == 2 else None
        auc = roc_auc_score(y_te, probs) if probs is not None else np.nan
        results.append({
            'feature': feat,
            'accuracy': accuracy_score(y_te, y_pred),
            'precision': precision_score(y_te, y_pred, zero_division=0),
            'recall': recall_score(y_te, y_pred, zero_division=0),
            'f1': f1_score(y_te, y_pred, zero_division=0),
            'roc_auc': auc
        })

    return pd.DataFrame(results).sort_values(by='accuracy', ascending=False).reset_index(drop=True)

In [24]:
single_results = run_log_single(df_train, target='label')
single_results

,feature,accuracy,precision,recall,f1,roc_auc
0,URLSimilarityIndex,0.996335,1.000000,0.991438,0.995701,0.996604
1,LineOfCode,0.949473,0.919782,0.966249,0.942443,0.990044
2,NoOfExternalRef,0.947807,0.904393,0.981886,0.941548,0.988241
3,NoOfSelfRef,0.935660,0.873623,0.993420,0.929678,0.972613
4,NoOfImage,0.927602,0.882882,0.957971,0.918895,0.979541
5,NoOfJS,0.902278,0.848890,0.938866,0.891614,0.970683
6,NoOfCSS,0.881952,0.845904,0.885587,0.865291,0.957214
7,HasSocialNet,0.877711,0.780071,0.994835,0.874460,0.892433
8,HasCopyrightInfo,0.864625,0.785107,0.941484,0.856214,0.874286
9,HasDescription,0.829517,0.729877,0.955353,0.827531,0.845334


### Try running logistic regression where we iteratively add in features

In [29]:
def run_log_incremental(df, target='label', predictors=None, test_size=0.2, random_state=42):
    from sklearn.model_selection import train_test_split
    from sklearn.linear_model import LogisticRegression
    from sklearn.preprocessing import StandardScaler
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
    import numpy as np
    import pandas as pd

    # Determine predictors if not provided
    if predictors is None:
        predictors = [c for c in df.columns if c != target]

    # Ensure 'URLSimilarityIndex' is added last if it exists
    if 'URLSimilarityIndex' in predictors:
        predictors = [f for f in predictors if f != 'URLSimilarityIndex'] + ['URLSimilarityIndex']

    y = df[target].copy()
    results = []
    selected_features = []

    for feat in predictors:
        selected_features.append(feat)
        X = df[selected_features].copy()

        # Train-test split
        X_tr, X_te, y_tr, y_te = train_test_split(
            X, y, test_size=test_size, stratify=y, random_state=random_state
        )

        # Standardize
        scaler = StandardScaler()
        X_tr_s = scaler.fit_transform(X_tr)
        X_te_s = scaler.transform(X_te)

        # Logistic regression
        m = LogisticRegression(solver='liblinear', max_iter=2000)
        m.fit(X_tr_s, y_tr)

        # Predictions
        y_pred = m.predict(X_te_s)
        probs = m.predict_proba(X_te_s)[:, 1] if len(m.classes_) == 2 else None

        auc = roc_auc_score(y_te, probs) if probs is not None else np.nan

        # Store metrics
        results.append({
            'features': selected_features.copy(),
            'num_features': len(selected_features),
            'accuracy': accuracy_score(y_te, y_pred),
            'precision': precision_score(y_te, y_pred, zero_division=0),
            'recall': recall_score(y_te, y_pred, zero_division=0),
            'f1': f1_score(y_te, y_pred, zero_division=0),
            'roc_auc': auc
        })

    # Return sorted DataFrame by number of features used
    return pd.DataFrame(results).sort_values(by='num_features').reset_index(drop=True)


In [30]:
results = run_log_incremental(df_train, target='label')
results

,features,num_features,accuracy,precision,recall,f1,roc_auc
0,[URLLength],1,0.743820,0.796180,0.539800,0.643390,0.731986
1,"[URLLength, DomainLength]",2,0.783866,0.914573,0.546169,0.683914,0.720715
2,"[URLLength, DomainLength, IsDomainIP]",3,0.783957,0.914604,0.546381,0.684089,0.721012
3,"[URLLength, DomainLength, IsDomainIP, CharContinuationRate]",4,0.796468,0.842290,0.645440,0.730842,0.764301
4,"[URLLength, DomainLength, IsDomainIP, CharContinuationRate, TLDLegitimateProb]",5,0.796165,0.840758,0.646289,0.730808,0.764706
5,"[URLLength, DomainLength, IsDomainIP, CharContinuationRate, TLDLegitimateProb, URLCharProb]",6,0.820611,0.866726,0.686549,0.766188,0.836677
6,"[URLLength, DomainLength, IsDomainIP, CharContinuationRate, TLDLegitimateProb, URLCharProb, TLDLength]",7,0.827941,0.873993,0.698861,0.776677,0.843786
7,"[URLLength, DomainLength, IsDomainIP, CharContinuationRate, TLDLegitimateProb, URLCharProb, TLDLength, NoOfSubDomain]",8,0.830032,0.873641,0.704946,0.780280,0.855732
8,"[URLLength, DomainLength, IsDomainIP, CharContinuationRate, TLDLegitimateProb, URLCharProb, TLDLength, NoOfSubDomain, HasObfuscation]",9,0.830032,0.873641,0.704946,0.780280,0.855745
9,"[URLLength, DomainLength, IsDomainIP, CharContinuationRate, TLDLegitimateProb, URLCharProb, TLDLength, NoOfSubDomain, HasObfuscation, NoOfObfuscatedChar]",10,0.830032,0.873641,0.704946,0.780280,0.855746


### Try running a logistic regression using only URL related features

In [28]:
URL_features = ["URLLength", "DomainLength", "IsDomainIP", "CharContinuationRate", "TLDLength", "NoOfSubDomain", "HasObfuscation",
                "NoOfObfuscatedChar", "ObfuscationRatio", "NoOfLettersInURL", "LetterRatioInURL", "NoOfDegitsInURL", "DegitRatioInURL",
                "NoOfEqualsInURL", "NoOfQMarkInURL", "NoOfAmpersandInURL", "NoOfOtherSpecialCharsInURL", "SpacialCharRatioInURL"]

In [31]:
results = run_log_incremental(df_train, predictors=URL_features, target='label')
results

,features,num_features,accuracy,precision,recall,f1,roc_auc
0,[URLLength],1,0.743820,0.796180,0.539800,0.643390,0.731986
1,"[URLLength, DomainLength]",2,0.783866,0.914573,0.546169,0.683914,0.720715
2,"[URLLength, DomainLength, IsDomainIP]",3,0.783957,0.914604,0.546381,0.684089,0.721012
3,"[URLLength, DomainLength, IsDomainIP, CharContinuationRate]",4,0.796468,0.842290,0.645440,0.730842,0.764301
4,"[URLLength, DomainLength, IsDomainIP, CharContinuationRate, TLDLength]",5,0.807131,0.862018,0.654214,0.743875,0.781478
5,"[URLLength, DomainLength, IsDomainIP, CharContinuationRate, TLDLength, NoOfSubDomain]",6,0.813583,0.869432,0.664332,0.753169,0.795432
6,"[URLLength, DomainLength, IsDomainIP, CharContinuationRate, TLDLength, NoOfSubDomain, HasObfuscation]",7,0.813583,0.869432,0.664332,0.753169,0.795430
7,"[URLLength, DomainLength, IsDomainIP, CharContinuationRate, TLDLength, NoOfSubDomain, HasObfuscation, NoOfObfuscatedChar]",8,0.813583,0.869432,0.664332,0.753169,0.795431
8,"[URLLength, DomainLength, IsDomainIP, CharContinuationRate, TLDLength, NoOfSubDomain, HasObfuscation, NoOfObfuscatedChar, ObfuscationRatio]",9,0.813583,0.869432,0.664332,0.753169,0.795434
9,"[URLLength, DomainLength, IsDomainIP, CharContinuationRate, TLDLength, NoOfSubDomain, HasObfuscation, NoOfObfuscatedChar, ObfuscationRatio, NoOfLettersInURL]",10,0.814037,0.869750,0.665252,0.753879,0.804725
